In [1]:
import cobra
import snek
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import straindesign as sd

# Build Model

## Include pDNA synthesis reaction

In [2]:
iML1515 = cobra.io.read_sbml_model('../models/iML1515.xml')
iML1515

Name,iML1515
Memory address,0x07f3a76ff5000
Number of metabolites,1877
Number of reactions,2712
Number of groups,0
Objective expression,1.0*BIOMASS_Ec_iML1515_core_75p37M - 1.0*BIOMASS_Ec_iML1515_core_75p37M_reverse_35685
Compartments,"cytosol, extracellular space, periplasm"


In [3]:
with iML1515 as tmp:
    # create the pDNA synthesis reaction
    pDNA_synthesis = cobra.Reaction('pDNA_synthesis')
    pDNA_synthesis.name = "Synthesis of Plasmid DNA"
    pDNA_synthesis.subsystem = 'Plasmid Production'
    pDNA_synthesis.lower_bound = 0
    pDNA_synthesis.upper_bound = 1000
    tmp.add_reaction(pDNA_synthesis)
    model1 = tmp.copy()

In [4]:
model1

Name,iML1515
Memory address,0x07f3a74c19540
Number of metabolites,1877
Number of reactions,2713
Number of groups,0
Objective expression,1.0*BIOMASS_Ec_iML1515_core_75p37M - 1.0*BIOMASS_Ec_iML1515_core_75p37M_reverse_35685
Compartments,"cytosol, extracellular space, periplasm"


## Include pDNA metabolite

In [5]:
pDNA_length   = 10 # base pairs
pDNA_sequence = 'ATCG'*round(pDNA_length/4)

AT_count = 0
CG_count = 0
for base in ['A','T','C','G']:
    nr = pDNA_sequence.count(base)
    if base in 'AT':
        AT_count += nr*2 # times 2 for the reverse strand
    elif base in 'CG':
        CG_count += nr*2 # times 2 for the reverse strand
    else:
        'Unknown Base'
        
print('Sequence          :',pDNA_sequence)
print('pDNA Length       :',pDNA_length,'base pairs')
print('Nr of dATP + dTTP :',AT_count,'nucleotides')
print('Nr of dCTP + dGTP :',CG_count,'nucleotides')

Sequence          : ATCGATCG
pDNA Length       : 10 base pairs
Nr of dATP + dTTP : 8 nucleotides
Nr of dCTP + dGTP : 8 nucleotides


In [6]:
with model1 as tmp:
    # create plasmid metabolite
    pDNA_c = cobra.Metabolite('pDNA_c')
    pDNA_c.formula = snek.elements.get_pDNA_formula(pDNA_sequence)
    pDNA_c.name = 'Synthetic Plasmid'
    pDNA_c.compartment = 'c'
    tmp.add_metabolites([pDNA_c])
    model2 = tmp.copy()

In [7]:
model2

Name,iML1515
Memory address,0x07f3a74b30be0
Number of metabolites,1878
Number of reactions,2713
Number of groups,0
Objective expression,1.0*BIOMASS_Ec_iML1515_core_75p37M - 1.0*BIOMASS_Ec_iML1515_core_75p37M_reverse_35685
Compartments,"cytosol, extracellular space, periplasm"


## Set Stoichiometry

In [8]:
with model2 as tmp:
    stoichiometry = {'datp_c':-AT_count/2,'dttp_c':-AT_count/2,'dctp_c':-CG_count/2,'dgtp_c':-CG_count/2}
    # 1.36 ATP per dNTP polymerization [1].
    # [1] Cunningham, D. S., Koepsel, R. R., Ataai, M. M., & Domach, M. M. (2009). 
    # Factors affecting plasmid production in Escherichia coli from a resource 
    # allocation standpoint. Microbial Cell Factories, 8(1), 1-17.
    nr_polymerization_reactions = AT_count + CG_count
    nr_ATP = nr_polymerization_reactions * 1.36
    stoichiometry['atp_c'] = - nr_ATP
    stoichiometry['adp_c'] =   nr_ATP
    stoichiometry['h_c']   =   nr_ATP - nr_polymerization_reactions
    stoichiometry['pi_c']  =   nr_ATP
    stoichiometry['ppi_c'] =   nr_polymerization_reactions
    stoichiometry['h2o_c'] = - nr_ATP
    stoichiometry['pDNA_c']=   1
    tmp.reactions.pDNA_synthesis.add_metabolites(stoichiometry)
    model3 = tmp.copy()
    
    # check for mass balance
    balance = model3.reactions.pDNA_synthesis.check_mass_balance()
    for element in balance:
        assert np.isclose(balance[element],0,atol=model2.tolerance,rtol=0), f'Not Mass Balanced {blance[element]}'

In [9]:
model3.reactions.pDNA_synthesis

Reaction identifier,pDNA_synthesis
Name,Synthesis of Plasmid DNA
Memory address,0x7f3a2eb41ff0
Stoichiometry,21.76 atp_c + 4.0 datp_c + 4.0 dctp_c + 4.0 dgtp_c + 4.0 dttp_c + 21.76 h2o_c --> 21.76 adp_c + 5.760000000000002 h_c + pDNA_c + 21.76 pi_c + 16 ppi_c 21.76 ATP C10H12N5O13P3 + 4.0 DATP C10H12N5O12P3 + 4.0 DCTP C9H12N3O13P3 + 4.0 DGTP C10H12N5O13P3 + 4.0 DTTP C10H13N2O14P3 + 21.76 H2O H2O --> 21.76 ADP C10H12N5O10P2 + 5.760000000000002 H+ +...
GPR,
Lower bound,0
Upper bound,1000


## Include pDNA sink reaction

In [10]:
with model3 as tmp:
    tmp.add_boundary(tmp.metabolites.pDNA_c,type='sink',lb=0,ub=1000)
    model4 = tmp.copy()

In [11]:
model4.sinks.SK_pDNA_c

Reaction identifier,SK_pDNA_c
Name,Synthetic Plasmid sink
Memory address,0x7f3a2dc04730
Stoichiometry,pDNA_c --> Synthetic Plasmid -->
GPR,
Lower bound,0
Upper bound,1000


## Save Model

In [12]:
model4.id = 'iML1515_pDNA'
model4.description = 'iML1515 with pDNA synthesis and sink reactions.'
# cobra.io.write_sbml_model(model4,'../models/iML1515_pDNA.xml')